In [15]:
import pystan


In [33]:
# 

code = '''
data {
    int<lower=1> N_SEG;
    int<lower=2> N;
    vector[N] y;
    int<lower=2> N_PRED;
}
parameters {
    vector[N_SEG] alpha;
    real beta;
    real<lower=0> sigma;
    vector[N_PRED] y_tilda;
} 
model {
    int m;
    for (n in 2:N) {
        m = (n-1)*N_SEG/N + 1;
        y[n] ~ normal(alpha[m]+ beta * y[n-1], sigma);
    }
    y_tilda[1] ~ normal(alpha[N_SEG]+ beta * y[N], sigma);
    for (i in 2:N_PRED) {
        y_tilda[i] ~ normal(alpha[N_SEG] + beta* y_tilda[i-1], sigma);
    }
}

'''
y =  [ 0.596,  2.127,  3.093,  0.423, -1.508,  3.28 , -5.571,  2.623,
        1.335, -1.234, -0.364,  0.081, -1.076, -0.791,  1.715,  1.266,
        0.566, -1.333,  1.871,  2.429, -2.064, -0.336,  1.407,  2.787,
        2.743, -3.073,  1.335,  1.163,  2.982,  4.317,  2.913,  0.432,
       -3.238,  1.785,  2.766,  2.252,  1.024,  2.167, -0.334, -0.106,
        1.959, -0.784,  0.387,  2.816, -0.762, -1.897,  1.99 , -0.965,
        2.714,  1.237,  0.701,  3.814, -1.459,  1.021,  4.114,  0.038,
       -1.391,  2.91 ]
data = {
    'N': len (y),
    'y': y,
    'N_SEG' : 1,
    'N_PRED' :4
}

fit = pystan.stan(model_code=code, data=data, iter=1000, chains=4)
# params = fs = fit.summary()['summary']
print fit

INFO:pystan:COMPILING THE C++ CODE FOR MODEL anon_model_ce0d6e651229326abb1570e7aad01c9f NOW.


Inference for Stan model: anon_model_ce0d6e651229326abb1570e7aad01c9f.
4 chains, each with iter=1000; warmup=500; thin=1; 
post-warmup draws per chain=500, total post-warmup draws=2000.

             mean se_mean     sd   2.5%    25%    50%    75%  97.5%  n_eff   Rhat
alpha[0]     0.93  6.7e-3    0.3   0.32   0.74   0.93   1.13   1.51 2000.0    1.0
beta        -0.16  3.1e-3   0.14  -0.44  -0.25  -0.16  -0.07   0.11 2000.0    1.0
sigma        2.05  4.6e-3    0.2    1.7   1.91   2.04   2.18   2.51 2000.0    1.0
y_tilda[0]   0.53    0.05   2.12  -3.77  -0.84   0.53   1.91   4.65 2000.0    1.0
y_tilda[1]   0.92    0.05   2.13  -3.24  -0.52   0.88   2.38   5.18 2000.0    1.0
y_tilda[2]   0.77    0.05   2.13  -3.35  -0.61   0.76   2.19   4.98 2000.0    1.0
y_tilda[3]   0.76    0.05   2.15  -3.61  -0.73   0.73    2.2   4.97 2000.0    1.0
lp__       -73.03    0.07   2.01 -78.04  -74.1 -72.67 -71.53 -70.15  850.0    1.0

Samples were drawn using NUTS at Fri Aug 11 21:10:33 2017.
For each parame

In [29]:
print fit

Inference for Stan model: anon_model_b3092487d10b6a3627bfdf9a48ac120e.
4 chains, each with iter=1000; warmup=500; thin=1; 
post-warmup draws per chain=500, total post-warmup draws=2000.

             mean se_mean     sd   2.5%    25%    50%    75%  97.5%  n_eff   Rhat
alpha[0]     0.53    0.01   0.49  -0.49   0.22   0.53   0.87   1.46 2000.0    1.0
alpha[1]     1.34    0.01   0.51   0.32   1.01   1.34   1.68    2.3 2000.0    1.0
alpha[2]     0.99    0.01   0.49  -0.02   0.67   0.98   1.33   1.95 2000.0    1.0
beta        -0.19  3.2e-3   0.14  -0.48  -0.28  -0.18  -0.09   0.09 2000.0    1.0
sigma        2.07  4.6e-3    0.2   1.71   1.92   2.05   2.19   2.52 2000.0    1.0
y_tilda[0]   0.52    0.05   2.28  -3.73  -1.08   0.52   2.06   5.04 2000.0    1.0
y_tilda[1]    0.9    0.05   2.26  -3.57   -0.6   0.93   2.35   5.42 2000.0    1.0
y_tilda[2]   0.83    0.05   2.19  -3.68  -0.61   0.91   2.23   5.27 2000.0    1.0
y_tilda[3]   0.83    0.05   2.21   -3.5   -0.6   0.86   2.27   5.18 2000.0 

In [ ]:

code = '''
data {
    int<lower=1> T;
    real y[T];
}
parameters {
    real mu;
    vector<lower = -1, upper = 1>[2] phi;
    vector<lower = -1, upper = 1>[2] theta;
    real<lower=0> sigma;
} 
model {
    vector[T] nu;
    vector[T] err;
    nu[1] = mu + phi[1] * mu;
    err[1] = y[1] - nu[1];
    nu[2] = mu + phi[1]*y[1] + theta[1]*err[1];
    err[2] = y[2] - nu[2];
    for (t in 3:T) {
        nu[t] = mu + phi[1] * y[t-1] + phi[2]*y[t-2] 
              + theta[1] * err[t-1] + theta[2] * err[t-2];
        err[t] = y[t] - nu[t];
    }
    mu ~ normal(0, 10);
    phi ~ normal(0, 2);
    theta ~ normal(0, 2);
    sigma ~ cauchy(0, 5);
    err ~ normal(0, sigma);
}
'''
y =  [ 0.596,  2.127,  3.093,  0.423, -1.508,  3.28 , -5.571,  2.623,
        1.335, -1.234, -0.364,  0.081, -1.076, -0.791,  1.715,  1.266,
        0.566, -1.333,  1.871,  2.429, -2.064, -0.336,  1.407,  2.787,
        2.743, -3.073,  1.335,  1.163,  2.982,  4.317,  2.913,  0.432,
       -3.238,  1.785,  2.766,  2.252,  1.024,  2.167, -0.334, -0.106,
        1.959, -0.784,  0.387,  2.816, -0.762, -1.897,  1.99 , -0.965,
        2.714,  1.237,  0.701,  3.814, -1.459,  1.021,  4.114,  0.038,
       -1.391,  2.91 ]
data = {
    'T': len (y),
    'y': y
}

fit = pystan.stan(model_code=code, data=data, iter=1000, chains=4)
# params = fs = fit.summary()['summary']
print fit

In [14]:

code = '''
data {
    int<lower=1> T;
    real y[T];
}
parameters {
    real mu;
    vector<lower = -1, upper = 1>[2] phi;
    vector<lower = -1, upper = 1>[2] theta;
    real<lower=0> sigma;
    real y_tilda;
} 
model {
    vector[T] nu;
    vector[T] err;
    nu[1] = mu + phi[1] * mu;
    err[1] = y[1] - nu[1];
    nu[2] = mu + phi[1]*y[1] + theta[1]*err[1];
    err[2] = y[2] - nu[2];
    for (t in 3:T) {
        nu[t] = mu + phi[1] * y[t-1] + phi[2]*y[t-2] 
              + theta[1] * err[t-1] + theta[2] * err[t-2];
        err[t] = y[t] - nu[t];
    }
    mu ~ normal(0, 10);
    phi ~ normal(0, 2);
    theta ~ normal(0, 2);
    sigma ~ cauchy(0, 5);
    err ~ normal(0, sigma);
    y_tilda ~ normal(mu + y[T]*phi[1]+y[T-1]*phi[2] + theta[1]*err[T] + theta[2]*err[T-1], sigma);
}

'''
y =  [ 2.352,  2.619,  3.292,  2.657,  0.739,  1.115,  0.148,  0.572,
        0.859,  1.081,  1.104,  0.952, -1.324, -0.998,  1.124, -0.442,
        0.824, -0.825, -2.64 , -0.563,  1.129,  1.047,  0.943,  0.781,
       -0.933,  0.124,  0.341,  1.1  , -0.282,  1.173,  0.089,  1.544,
        0.026, -0.855,  1.448,  0.426, -0.245,  1.346, -0.36 , -0.328,
        0.482,  0.614,  0.572,  0.737, -0.097,  1.668, -0.012,  0.97 ,
       -0.898,  0.159,  1.543, -0.297,  1.257,  0.761,  0.725,  0.629,
        1.871,  0.061]
data = {
    'T': len (y),
    'y': y
}

fit = pystan.stan(model_code=code, data=data, iter=1000, chains=4)
# params = fs = fit.summary()['summary']
print fit

INFO:pystan:COMPILING THE C++ CODE FOR MODEL anon_model_6abda944bb0b6acb21d3626f5be8dfef NOW.


Inference for Stan model: anon_model_6abda944bb0b6acb21d3626f5be8dfef.
4 chains, each with iter=1000; warmup=500; thin=1; 
post-warmup draws per chain=500, total post-warmup draws=2000.

           mean se_mean     sd   2.5%    25%    50%    75%  97.5%  n_eff   Rhat
mu         0.61  9.2e-3   0.26   0.17   0.44    0.6   0.76   1.16  766.0    1.0
phi[0]     0.33    0.02   0.34  -0.46   0.14   0.41   0.57   0.85  295.0   1.02
phi[1]    -0.45    0.02   0.32   -0.9  -0.67  -0.51  -0.28   0.33  262.0   1.03
theta[0]   -0.1    0.02    0.3  -0.59  -0.31  -0.16   0.06   0.62  300.0   1.02
theta[1]    0.6    0.01   0.31  -0.18   0.46   0.68   0.84   0.97  450.0   1.02
sigma      1.01  2.8e-3    0.1   0.84   0.94    1.0   1.07   1.22 1223.0    1.0
y_tilda    0.62    0.03   1.08  -1.45  -0.09   0.63   1.31   2.82 1249.0    1.0
lp__     -33.79    0.09   2.05 -38.48 -34.94  -33.4 -32.31 -30.78  523.0    1.0

Samples were drawn using NUTS at Thu Aug 10 22:57:51 2017.
For each parameter, n_eff is a cr